In [ ]:
####
# Creator: Rowan Converse (rowanconverse@unm.edu)
# Date: 2022/08/02
# Purpose: Scratch, testing pieces of the full code to translate raw labels generated by volunteers on the Zooniverse platform into COCO format
# Ref COCO Camera Trap Standard: https://cocodataset.org/#format-data
####

In [3]:
#Loading JSON within CSV
import pandas as pd
import json
import datetime
from collections import OrderedDict 

path = r"/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/zooniverse_snippet.csv"
zooniverse = pd.read_csv(path)

In [ ]:
#Benni's example: load first row's annotations
row0_anno = json.loads(zooniverse.annotations[0])
print('Row 0: number of annotations = {}'.format(len(row0_anno)))

first_anno = row0_anno[0]['value'][0]    # extract the first of two annotations of the first row, then its properties under 'value' (dict), and again the first entry (that property is a list)
print('First annotation: bbox = {}, {}, {}, {}; label = {}'.format(
    first_anno['x'], first_anno['y'],
    first_anno['width'], first_anno['height'],
    first_anno['tool_label']
))

In [19]:
# Statistics about annotations
row0_anno = json.loads(zooniverse.annotations[0])
first_anno = row0_anno[0]['value'][0]
print(first_anno)

#annotations = []
#for i in range(len(data)):
#       for x in data[i]["value"]:
#              annotations.append(x)

#print(len(annotations))

{'x': 575.9765625, 'y': 265.39453125, 'tool': 0, 'frame': 0, 'width': 79.1796875, 'height': 94.40625, 'details': [], 'tool_label': 'Duck'}


TypeError: the JSON object must be str, bytes or bytearray, not Series

In [23]:
#Statistics about images

images = zooniverse.subject_ids
print("Number of Images: "+str(len(images.unique())))

#print('Animal images: '+str(len([ann['value'] for ann in anno if ann['value'] != 0])))
#print('Empty images: '+str(len([ann['value'] for ann in anno if ann['value'] == 0])))

Number of Images: 282


In [106]:
subj = json.loads(zooniverse.subject_data[0])
for val in subj.values():
   for item in val:
        print(item)
     

#imglist = list(set(img["subject_data"]['Filename'] for img in zooniverse))
#imglist
#Add unique IDs to each filename
#list_ids = [{v: k for k, v in enumerate(
#   OrderedDict.fromkeys(imglist), 1)}
#      [n] for n in imglist]
#img = dict(zip(list_ids, imglist))

retired
Filename


In [51]:
#Statistics about labelers
userlist = list(set(zooniverse.user_id))

userIDs = [{v: k for k, v in enumerate(
   OrderedDict.fromkeys(userlist), 1)}
      [n] for n in userlist]
labelers = dict(zip(userIDs, userlist))
labelers
#print("Number of Labelers: "+str(len(users.unique())))

{1: 2313184,
 2: 2263329,
 3: 1822173,
 4: 2316323,
 5: 2141263,
 6: 2247823,
 7: 2313201,
 8: 2313202,
 9: 2154838,
 10: 2091290,
 11: 2142076,
 12: 2316349,
 13: 2316350}

In [74]:
row10 = json.loads(zooniverse.annotations[10])
an0row10 = row10[0]['value'][0]
duck = an0row10["tool_label"] 
print(duck)

Duck


In [114]:
#Categories
spplist = []
for i in range(len(zooniverse)):
  row = json.loads(zooniverse.annotations[i])
  for j in range(len(row)):
    annlist = row[j]['value']
    for k in range(len(annlist)):
      ann = annlist[k]
      cur_species = ann["tool_label"]
      spplist.append(cur_species)
spp = set(spplist)

sppIDs = [{v: k for k, v in enumerate(
   OrderedDict.fromkeys(spplist), 1)}
      [n] for n in spp]
cat = dict(zip(sppIDs, spp))
categories = {"categories": cat}
categories

TypeError: string indices must be integers

In [120]:
#Annotations-- 
for i in range(len(zooniverse)):
  row = json.loads(zooniverse.annotations[i])
  for j in range(len(row)):
    if row[j]['task'] != 'T1':
        # task was not to draw a bounding box
        continue
    annlist = row[j]['value']
    for k in range(len(annlist)):
      ann = annlist[k]
      x = ann["x"]
      y = ann["y"]
      w = ann["width"]
      h = ann["height"]
      label = ann["tool_label"]
      anno = [label, x, y, w, h]
      print(anno)
      
#spp = set(spplist)

['Duck', 575.9765625, 265.39453125, 79.1796875, 94.40625]
['Duck', 185.109375, 124.38671875, 62.6328125, 30.52734375]
['Duck', 5.06640625, 223.09765625, 42.37109375, 51.4453125]
['Duck', 1.75390625, 317.02734375, 53.32421875, 56.390625]
['Duck', 368.42578125, 149.796875, 61.16015625, 36.3046875]
['Duck', 322.69921875, 39.5390625, 81.23046875, 34.07421875]
['Duck', 509.6328125, 189.3984375, 70.2265625, 31.53515625]
['Duck', 490.140625, 261.484375, 76.5390625, 27.76171875]
['Duck', 402.88671875, 329.359375, 90.72265625, 40.3671875]
['Duck', 325.5478210449219, 270.52587890625, 73.85104370117188, 72.52145385742188]
['Duck', 336.7303161621094, 401.5649719238281, 61.29571533203125, 93.30828857421875]
['Duck', 207.77734375, 2.83984375, 49.7421875, 16.66015625]
['Duck', 225.41796875, 52.8046875, 32.66796875, 32.36328125]
['Duck', 269.51171875, 37.29296875, 30.5234375, 32.2265625]
['Duck', 458.16015625, 3.140625, 16.49609375, 9.37890625]
['Duck', 453.125, 40.58984375, 29.5078125, 39.984375]
['D